In [2]:
import numpy as np
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
np.load('./patch_points_cloud.npy')

array([[ 7.17216773,  1.56518707,  3.56518707],
       [ 7.17091603,  1.540074  ,  3.56491391],
       [ 7.16966433,  1.51496961,  3.56464075],
       ...,
       [ 8.39936783, -1.80390455,  0.13790498],
       [ 8.39936783, -1.83299978,  0.13790498],
       [ 8.39936783, -1.86209502,  0.13790498]])

In [9]:
pcd = o3d.io.read_point_cloud('o3d_world_points_cloud.pcd')
pcd_patch = o3d.io.read_point_cloud('o3d_patch_points_cloud.pcd')

In [3]:
import torch
def get_patch_data(file_path: str):
    patch = torch.load(file_path, map_location='cpu')

    return patch.permute((1, 2, 0)).detach().numpy()

patch = get_patch_data('../patch.pt')

In [10]:
import numpy as np
import open3d as o3d
# from pyntcloud import PyntCloud

# points = pos_3d[:3].numpy().reshape((-1, 3), order='F')
# points = point_clouds[81][:3].transpose()
# point_cloud.farthest_point_down_sample(num_samples=1000)
o3d.visualization.draw_geometries([pcd_patch, pcd])

In [3]:
import carla
import random
import queue

import numpy
import numpy as np

from numpy.matlib import repmat
import cv2
import open3d as o3d
# from image_converter import *

import math


def to_bgra_array(image):
    """Convert a CARLA raw image to a BGRA numpy array."""
    if not isinstance(image, carla.Image):
        raise ValueError("Argument must be a carla.sensor.Image")
    array = numpy.frombuffer(image.raw_data, dtype=numpy.dtype("uint8"))
    array = numpy.reshape(array, (image.height, image.width, 4))

    return array


def depth_to_array(image):
    """
    Convert an image containing CARLA encoded depth-map to a 2D array containing
    the depth value of each pixel normalized between [0.0, 1.0].
    """
    array = to_bgra_array(image)
    array = array.astype(numpy.float32)
    # Apply (R + G * 256 + B * 256 * 256) / (256 * 256 * 256 - 1).
    normalized_depth = numpy.dot(array[:, :, :3], [65536.0, 256.0, 1.0])
    normalized_depth /= 16777215.0  # (256.0 * 256.0 * 256.0 - 1.0)

    return normalized_depth


def depth_to_local_point_cloud(image, color=None, max_depth=0.9):
    """
    Convert an image containing CARLA encoded depth-map to a 2D array containing
    the 3D position (relative to the camera) of each pixel and its corresponding
    RGB color of an array.
    "max_depth" is used to omit the points that are far enough.
    """
    far = 1000.0  # max depth in meters.
    normalized_depth = depth_to_array(image)

    # (Intrinsic) K Matrix
    k = numpy.identity(3)
    k[0, 2] = image.width / 2.0
    k[1, 2] = image.height / 2.0
    k[0, 0] = k[1, 1] = image.width / (2.0 * math.tan(image.fov * math.pi / 360.0))

    # 2d pixel coordinates
    pixel_length = image.width * image.height
    u_coord = repmat(numpy.r_[image.width-1:-1:-1],
                     image.height, 1).reshape(pixel_length)
    v_coord = repmat(numpy.c_[image.height-1:-1:-1],
                     1, image.width).reshape(pixel_length)
    if color is not None:
        color = color.reshape(pixel_length, 3)
    normalized_depth = numpy.reshape(normalized_depth, pixel_length)

    # Search for pixels where the depth is greater than max_depth to
    # delete them
    max_depth_indexes = numpy.where(normalized_depth > max_depth)
    normalized_depth = numpy.delete(normalized_depth, max_depth_indexes)
    u_coord = numpy.delete(u_coord, max_depth_indexes)
    v_coord = numpy.delete(v_coord, max_depth_indexes)
    if color is not None:
        color = numpy.delete(color, max_depth_indexes, axis=0)

    # pd2 = [u,v,1]
    p2d = numpy.array([u_coord, v_coord, numpy.ones_like(u_coord)])

    # P = [X,Y,Z]
    p3d = numpy.dot(numpy.linalg.inv(k), p2d)

    p3d *= normalized_depth * far
    p3d = numpy.concatenate((p3d, numpy.ones((1, p3d.shape[1]))))

    return p3d, color


def get_camera2world_matrix(carla_transform: carla.Transform, real_y_axis=False):
    """
    Args:
        carla_transform: Carla.Transform instance, contains carla.Location and carla.Rotation
        real_y_axis: return real y-axis value when setting true. but the view of point cloud in open-3d
            will be reversed in yaw direction.
    Returns:
        a 4x4 rotation & transaction matrix that transforms coords from camera coord-sys to simu-world coord-sys.
    """
    camera2vehicle_matrix = np.array([[0, 0, 1, 0], [1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 0, 1]], dtype=np.float64)

    pitch = carla_transform.rotation.pitch / 180.0 * math.pi
    yaw = carla_transform.rotation.yaw / 180.0 * math.pi
    roll = carla_transform.rotation.roll / 180.0 * math.pi
    loc_x = carla_transform.location.x
    loc_y = - carla_transform.location.y
    loc_z = carla_transform.location.z
    sin_y, sin_p, sin_r = math.sin(yaw), math.sin(pitch), math.sin(roll)
    cos_y, cos_p, cos_r = math.cos(yaw), math.cos(pitch), math.cos(roll)

    vehicle2world_matrix = np.array([
        [cos_y * cos_p, cos_y * sin_p * sin_r + sin_y * cos_r, - cos_y * sin_p * cos_r + sin_y * sin_r, loc_x],
        [-sin_y * cos_p, - sin_y * sin_p * sin_r + cos_y * cos_r, sin_y * sin_p * cos_r + cos_y * sin_r, loc_y],
        [sin_p, -cos_p * sin_r, cos_p * cos_r, loc_z],
        [0.0, 0.0, 0.0, 1.0]
    ])
    if real_y_axis:
        vehicle2world_matrix[1] *= -1

    return vehicle2world_matrix @ camera2vehicle_matrix


def generate_lidar_bp(blueprint_library, delta):
    """
    To get lidar bp
    :param blueprint_library: the world blueprint_library
    :param delta: update rate(s)
    :return: lidar bp
    """
    lidar_bp = blueprint_library.find("sensor.lidar.ray_cast")
    lidar_bp.set_attribute("dropoff_general_rate", "0.0")
    lidar_bp.set_attribute("dropoff_intensity_limit", "1.0")
    lidar_bp.set_attribute("dropoff_zero_intensity", "0.0")

    lidar_bp.set_attribute("upper_fov", str(15.0))
    lidar_bp.set_attribute("lower_fov", str(-25.0))
    lidar_bp.set_attribute("channels", str(64.0))
    lidar_bp.set_attribute("range", str(100.0))
    lidar_bp.set_attribute("rotation_frequency", str(1.0 / delta))
    lidar_bp.set_attribute("points_per_second", str(500000))

    return lidar_bp

#连接Carla并获取世界
client = carla.Client('localhost', 2000)
world  = client.get_world()
bp_lib = world.get_blueprint_library()

# 生成车辆
vehicle_bp =bp_lib.find('vehicle.lincoln.mkz_2020')
spawn_points = world.get_map().get_spawn_points()
vehicle = world.try_spawn_actor(vehicle_bp, spawn_points[0])

# 生成相机
rgb_camera_bp = bp_lib.find('sensor.camera.rgb')
rgb_camera_bp.set_attribute('fov', '90')

rgb_camera_init_trans = carla.Transform(carla.Location(z=2))
rgb_camera = world.spawn_actor(rgb_camera_bp, rgb_camera_init_trans, attach_to=vehicle)

depth_camera_bp = bp_lib.find('sensor.camera.depth')
depth_camera_bp.set_attribute('fov', '90')

depth_camera_init_trans = carla.Transform(carla.Location(z=2))
depth_camera = world.spawn_actor(depth_camera_bp, depth_camera_init_trans, attach_to=vehicle, attachment_type=carla.AttachmentType.Rigid)

lidar_bp = generate_lidar_bp(bp_lib, delta=0.05)
lidar_camera_init_trans = carla.Transform(carla.Location(z=2))
lidar_camera = world.spawn_actor(depth_camera_bp, lidar_camera_init_trans, attach_to=vehicle)

# 设置traffic manager
tm = client.get_trafficmanager(8000)
tm.set_synchronous_mode(True)
# 是否忽略红绿灯
tm.ignore_lights_percentage(vehicle, 100)
# 如果限速30km/h -> 30*(1-10%)=27km/h
# tm.global_percentage_speed_difference(10.0)


# 设置自动驾驶
# vehicle.set_autopilot(True)
vehicle.set_autopilot(True, tm.get_port())

# 设置仿真模式为同步模式
settings = world.get_settings()
settings.synchronous_mode = True # 启用同步模式
settings.fixed_delta_seconds = 0.05
world.apply_settings(settings)

# 创建对接接收相机数据
depth_image_queue = queue.Queue()
depth_camera.listen(depth_image_queue.put)

rgb_image_queue = queue.Queue()
rgb_camera.listen(rgb_image_queue.put)

lidar_queue = queue.Queue()
lidar_camera.listen(lidar_queue.put)

# 从相机获取属性
image_w = rgb_camera_bp.get_attribute("image_size_x").as_int()  # 图像宽度
image_h = rgb_camera_bp.get_attribute("image_size_y").as_int()  # 图像高度
fov = rgb_camera_bp.get_attribute("fov").as_float()  # 视场角


# 获取第一张图像
world.tick()
depth_image = depth_image_queue.get()
rgb_image = rgb_image_queue.get()
# 将原始数据重新整形为 RGB 数组
rgb_img = np.reshape(np.copy(rgb_image.raw_data), (rgb_image.height, rgb_image.width, 4))

# 在 OpenCV 的显示窗口中显示图像
cv2.namedWindow('ImageWindowName', cv2.WINDOW_AUTOSIZE)
cv2.imshow('ImageWindowName', rgb_img)
cv2.waitKey(1)

point_clouds = []
colors = []

idx = -1

prj_config = []

while True:
    # 更新世界状态并获取图像
    world.tick()
    idx += 1

    depth_image = depth_image_queue.get()
    rgb_image = rgb_image_queue.get()
    rgb_image = np.reshape(np.copy(rgb_image.raw_data), (rgb_image.height, rgb_image.width, 4))

    lidar_data = lidar_queue.get()

    if idx % 40 == 0:

        # Tvc_matrix = np.array([[0, 0, 1, 0], [1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 0, 1]])
        Tvc_matrix = np.array([[0, 0, 1, 0], [1, 0, 0, 0], [0, -1, 0, 0], [0, 0, 0, 1]])
        # Tvc_matrix = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])

        # transform = depth_camera.get_transform()
        #
        # pitch = transform.rotation.pitch / 180.0 * math.pi
        # yaw = transform.rotation.yaw / 180.0 * math.pi
        # roll = transform.rotation.roll / 180.0 * math.pi
        # loc_x, loc_y, loc_z = transform.location.x, - transform.location.y, transform.location.z
        # sin_y, sin_p, sin_r = math.sin(yaw), math.sin(pitch), math.sin(roll)
        # cos_y, cos_p, cos_r = math.cos(yaw), math.cos(pitch), math.cos(roll)
        #
        # camera_to_world = np.array(
        #     [[cos_y * cos_p, cos_y * sin_p * sin_r + sin_y * cos_r, - cos_y * sin_p * cos_r + sin_y * sin_r, loc_x],
        #      [-sin_y * cos_p, - sin_y * sin_p * sin_r + cos_y * cos_r, sin_y * sin_p * cos_r + cos_y * sin_r, loc_y],
        #      [sin_p, -cos_p * sin_r, cos_p * cos_r, loc_z],
        #      [0, 0, 0, 1]]
        # )

        # rx = np.array([
        #     [1, 0, 0], [0, math.cos(roll), - math.sin(roll)], [0, math.sin(roll), math.cos(roll)],
        # ], dtype=np.float64)
        # ry = np.array([
        #     [math.cos(pitch), 0, math.sin(pitch)], [0, 1, 0], [- math.sin(pitch), 0, math.cos(pitch)]
        # ], dtype=np.float64)
        # rz = np.array([
        #     [math.cos(yaw), - math.sin(yaw), 0], [math.sin(yaw),  math.cos(yaw), 0], [0, 0, 1]
        # ], dtype=np.float64)

        # mat = transform.get_matrix()
        # x, y, z = depth_camera.get_transform().location.x, - depth_camera.get_transform().location.y, depth_camera.get_transform().location.z
        # yaw, pitch, roll = - depth_camera.get_transform().rotation.yaw, - depth_camera.get_transform().rotation.pitch, - depth_camera.get_transform().rotation.roll
        # transform = carla.Transform(carla.Location(x=x, y=y, z=z), carla.Rotation(yaw=yaw, pitch=pitch, roll=roll))
        # mat = transform.get_matrix()

        p3d, color = depth_to_local_point_cloud(depth_image, rgb_image[:, :, :3], max_depth=0.8)

        # p3d = camera_to_world @ Tvc_matrix @ p3d[:3] + np.array([[loc_x], [loc_y], [loc_z]])
        # camera_to_world[1] *= -1
        # p3d = (camera_to_world @ Tvc_matrix @ p3d)[:3]
        mat = get_camera2world_matrix(depth_camera.get_transform())

        p3d = mat @ p3d

        # print('car pos:', (depth_camera.get_transform().location.x, depth_camera.get_transform().location.y, depth_camera.get_transform().location.z))
        # print('pix pos:', p3d[:, 0])
        # p3d = rz @ ry @ rx @ (Tvc_matrix @ p3d)[:3] + np.array([[loc_x], [loc_y], [loc_z]])

        point_clouds.append(p3d[:3])
        colors.append(color)

    cv2.imshow('ImageWindowName', rgb_image)
    if cv2.waitKey(1) == ord('q'):
        break

cv2.destroyAllWindows()

depth_camera.destroy()
rgb_camera.destroy()
lidar_camera.destroy()
vehicle.destroy()
depth_image_queue.queue.clear()
rgb_image_queue.queue.clear()
lidar_queue.queue.clear()

In [2]:
np.array(np.broadcast_to(depth_to_array(prj_config[0]).reshape(600, 800, 1), (600, 800, 3)) * 255.0, dtype=np.uint8)

IndexError: list index out of range

## $轴的逆变换：\begin{bmatrix}\ 0 & 0 & 1 & 0\\ 1 & 0 & 0 & 0\\ 0 & -1 & 0 & 0\\ 0 & 0 & 0 & 1 \end{bmatrix} $ ##

# $
\begin{align*}
  \begin{bmatrix}
x^{'} \\ y^{'} \\ z^{'}
\end{bmatrix}
&=
\begin{bmatrix}
x_{0} \\ y_{0} \\ z_{0}
\end{bmatrix}
+
\begin{bmatrix}
cos(-yaw) & -sin(-yaw) & 0
\\ sin(-yaw) & cos(-yaw) & 0
\\ 0 & 0 & 1\end{bmatrix}
\begin{bmatrix}
cos(-pitch) & 0 & sin(-pitch)
\\ 0 & 1 & 0
\\ -sin(-pitch) & 0 & cos(-pitch)
\end{bmatrix}
\begin{bmatrix}
1 & 0 & 0
\\ 0 & cos(-roll) & -sin(-roll)
\\ 0 & sin(-roll) & cos(-roll)
\end{bmatrix}
\begin{bmatrix}
x \\ y \\ z
\end{bmatrix} \\

&=
\begin{bmatrix}
x_{0} \\ y_{0} \\ z_{0}
\end{bmatrix}
+
\begin{bmatrix}
cos(y)cos(p) & cos(y)sin(p)sin(r) + sin(y)cos(r) & -cos(y)sin(p)cos(r)+sin(y)sin(r)
\\-sin(y)cos(p) & -sin(y)sin(p)sin(r)+cos(y)cos(r) & sin(y)sin(p)cos(r)+cos(y)sin(r)
\\ sin(p) & -cos(p)sin(r) & cos(p)cos(r)
\end{bmatrix}
\begin{bmatrix}
x \\ y \\ z
\end{bmatrix}
\\
\end{align*}
$
# $
\iff
\begin{bmatrix}
x^{'} \\ y^{'} \\ z^{'} \\ 1
\end{bmatrix}
=
\begin{bmatrix}
c_y \cdot c_p & c_y \cdot s_p \cdot s_r + s_y \cdot c_r & - c_y \cdot s_p \cdot c_r + s_y \cdot s_r & x_{0}
\\-s_y \cdot c_p & -s_y \cdot s_p \cdot s_r + c_y \cdot c_r & s_y \cdot s_p \cdot c_r + c_y \cdot s_r & y_{0}
\\ s_p & -c_p \cdot s_r & c_p \cdot c_r & z_{0}
\\ 0 & 0 & 0 & 1
\end{bmatrix}
\begin{bmatrix}
x \\ y \\ z \\ 1
\end{bmatrix}
$

# $
\begin{bmatrix}
x_{global} \\ -y_{global} \\ z_{global}
\end{bmatrix}
+
\begin{bmatrix}
cos(-yaw) & -sin(-yaw) & 0
\\ sin(-yaw) & cos(-yaw) & 0
\\ 0 & 0 & 1\end{bmatrix}
\begin{bmatrix}
cos(-pitch) & 0 & sin(-pitch)
\\ 0 & 1 & 0
\\ -sin(-pitch) & 0 & cos(-pitch)
\end{bmatrix}
\begin{bmatrix}
1 & 0 & 0
\\ 0 & cos(-roll) & -sin(-roll)
\\ 0 & sin(-roll) & cos(-roll)
\end{bmatrix}
\begin{bmatrix}
0 & 0 & 1
\\ 1 & 0 & 0 \\ 0 & 1 & 0
\end{bmatrix}
\begin{bmatrix}
x_{camera} \\ y_{camera} \\ z_{camera}
\end{bmatrix}
=
\begin{bmatrix}
x_{global} + x_{camera} \cdot cos(y)cos(p) + y_{camera} \cdot (cos(y)sin(p)sin(r) + sin(y)cos(r)) + z_{camera} \cdot (-cos(y)sin(p)cos(r)+sin(y)sin(r))
\\ -y_{global} + x_{camera} \cdot -sin(y)cos(p) + y_{camera} \cdot (-sin(y)sin(p)sin(r)+cos(y)cos(r)) + z_{camera} \cdot (sin(y)sin(p)cos(r)+cos(y)sin(r))
\\ z_{global} + x_{camera} \cdot sin(p) + y_{camera} \cdot (-cos(p)sin(r)) + z_{camera} \cdot (cos(p)cos(r))
\end{bmatrix}
$ #

In [ ]:
prj_config[1].x, prj_config[1].y, prj_config[1].z

In [ ]:
p3d[:, 6]

In [ ]:
def prj_patch2point_cloud(depth_data: carla.Image, patch_data: numpy.ndarray, camera_location: carla.Location, camera_rotation: carla.Rotation, max_depth=0.9, min_depth=0.0):
    depth_array = depth_to_array(depth_data)

    far = 1000.0
    k = numpy.identity(3)
    cw, ch = depth_data.width / 2.0, depth_data.height / 2.0
    k[0, 2], k[1, 2] = cw, ch
    k[0, 0] = k[1, 1] = depth_data.width / (2.0 * math.tan(depth_data.fov * math.pi / 360.0))

    print(depth_array.shape)
    # patch_depth = depth_array[
    #               int(ch) - patch_data.shape[0] // 2: int(ch) - patch_data.shape[0] // 2 + patch_data.shape[0],
    #               int(cw) - patch_data.shape[1] // 2: int(cw) - patch_data.shape[1] // 2 + patch_data.shape[1]]
    patch_depth = depth_array[
              0 : patch_data.shape[0],
              0 : patch_data.shape[1]]

    pixel_length = patch_data.shape[0] * patch_data.shape[1]
    u_coord = repmat(numpy.r_[patch_data.shape[1] - 1:-1:-1], patch_data.shape[0], 1).reshape(pixel_length)
    v_coord = repmat(numpy.c_[patch_data.shape[0] - 1:-1:-1], 1, patch_data.shape[1]).reshape(pixel_length)

    color = patch_data.reshape(pixel_length, 3)
    normalized_depth = numpy.reshape(patch_depth, pixel_length)

    # Search for pixels where the depth is greater than max_depth to
    # delete them
    max_depth_indexes = numpy.where(normalized_depth > max_depth)
    normalized_depth = numpy.delete(normalized_depth, max_depth_indexes)
    u_coord = numpy.delete(u_coord, max_depth_indexes)
    v_coord = numpy.delete(v_coord, max_depth_indexes)
    if color is not None:
        color = numpy.delete(color, max_depth_indexes, axis=0)

    # pd2 = [u,v,1]
    p2d = numpy.array([u_coord, v_coord, numpy.ones_like(u_coord)])

    # P = [X,Y,Z]
    p3d = numpy.dot(numpy.linalg.inv(k), p2d)

    p3d *= normalized_depth * far
    p3d = numpy.concatenate((p3d, numpy.ones((1, p3d.shape[1]))))

    Tvc_matrix = np.array([[0, 0, 1], [1, 0, 0], [0, 1, 0]])

    pitch = camera_rotation.pitch / 180.0 * math.pi
    yaw = camera_rotation.yaw / 180.0 * math.pi
    roll = camera_rotation.roll / 180.0 * math.pi
    loc_x, loc_y, loc_z = camera_location.x, - camera_location.y, camera_location.z
    sin_y, sin_p, sin_r = math.sin(yaw), math.sin(pitch), math.sin(roll)
    cos_y, cos_p, cos_r = math.cos(yaw), math.cos(pitch), math.cos(roll)

    camera_to_world = np.array(
        [[cos_y * cos_p, cos_y * sin_p * sin_r + sin_y * cos_r, - cos_y * sin_p * cos_r + sin_y * sin_r],
         [-sin_y * cos_p, - sin_y * sin_p * sin_r + cos_y * cos_r, sin_y * sin_p * cos_r + cos_y * sin_r],
         [sin_p, -cos_p * sin_r, cos_p * cos_r]]
    )

    p3d = camera_to_world @ Tvc_matrix @ p3d[:3] + np.array([[loc_x], [loc_y], [loc_z]])

    return p3d, color

In [ ]:
cv2.imshow('cc', depth_to_array(prj_config[0]) * 255)

In [ ]:
#####################################################
# To acquire point cloud and display it with open3d
#####################################################
import sys
import time

import carla
import open3d as o3d
import numpy as np
import random
from matplotlib import cm
from datetime import datetime

VIDIDIS = np.array(cm.get_cmap("plasma").colors)
VID_RANGE = np.linspace(0.0, 1.0, VIDIDIS.shape[0])

def generate_lidar_bp(blueprint_library, delta):
    """
    To get lidar bp
    :param blueprint_library: the world blueprint_library
    :param delta: update rate(s)
    :return: lidar bp
    """
    lidar_bp = blueprint_library.find("sensor.lidar.ray_cast")
    lidar_bp.set_attribute("dropoff_general_rate", "0.0")
    lidar_bp.set_attribute("dropoff_intensity_limit", "1.0")
    lidar_bp.set_attribute("dropoff_zero_intensity", "0.0")

    lidar_bp.set_attribute("upper_fov", str(15.0))
    lidar_bp.set_attribute("lower_fov", str(-25.0))
    lidar_bp.set_attribute("channels", str(64.0))
    lidar_bp.set_attribute("range", str(100.0))
    lidar_bp.set_attribute("rotation_frequency", str(1.0 / delta))
    lidar_bp.set_attribute("points_per_second", str(500000))

    return lidar_bp


def lidar_callback(point_cloud, point_list):
    # We need to convert point cloud(carla-format) into numpy.ndarray
    data = np.copy(np.frombuffer(point_cloud.raw_data, dtype = np.dtype("f4")))
    data = np.reshape(data, (int(data.shape[0] / 4), 4))

    intensity = data[:, -1]
    intensity_col = 1.0 - np.log(intensity) / np.log(np.exp(-0.004 * 100))
    int_color = np.c_[
        np.interp(intensity_col, VID_RANGE, VIDIDIS[:, 0]),
        np.interp(intensity_col, VID_RANGE, VIDIDIS[:, 1]),
        np.interp(intensity_col, VID_RANGE, VIDIDIS[:, 2])]

    points = data[:, :-1] # we only use x, y, z coordinates
    points[:, 1] = -points[:, 1] # This is different from official script
    point_list.points = o3d.utility.Vector3dVector(points)
    point_list.colors = o3d.utility.Vector3dVector(int_color)


if __name__ == "__main__":
    print(f"Let's show point cloud with open3d in carla!")
    client = carla.Client("127.0.0.1", 2000)
    client.set_timeout(2.0)
    world = client.get_world()

    try:
        # 1. To do some synchronous settings in world
        original_settings = world.get_settings()
        settings = world.get_settings()
        traffic_manager = client.get_trafficmanager(8000)
        traffic_manager.set_synchronous_mode(True)

        delta = 0.05

        settings.fixed_delta_seconds = delta
        settings.synchronous_mode = True
        # settings.no_rendering_mode = True
        world.apply_settings(settings)

        # 2. To get blueprint for your ego vehicle and spawn it!
        blueprint_library = world.get_blueprint_library()
        vehicle_bp = blueprint_library.filter("model3")[0]
        vehicle_transform = random.choice(world.get_map().get_spawn_points())
        vehicle = world.spawn_actor(vehicle_bp, vehicle_transform)
        vehicle.set_autopilot(True)

        # 3. To get lidar blueprint and spawn it on your car!
        lidar_bp = generate_lidar_bp(blueprint_library, delta)
        lidar_transform = carla.Transform(carla.Location(x = -0.5, z = 1.8))
        lidar = world.spawn_actor(lidar_bp, lidar_transform, attach_to = vehicle)

        # 4. We set a point_list to store our point cloud
        point_list = o3d.geometry.PointCloud()

        # 5. Listen to the lidar to collect point cloud
        lidar.listen(lambda data: lidar_callback(data, point_list))

        # 6. We set some basic settings for display with open3d
        vis = o3d.visualization.Visualizer()
        vis.create_window(
            window_name= "Display Point Cloud",
            width= 960,
            height= 540,
            left= 480,
            top= 270)

        vis.get_render_option().background_color = [0.05, 0.05, 0.05]
        vis.get_render_option().point_size = 1
        vis.get_render_option().show_coordinate_frame = True

        frame = 0
        dt0 = datetime.now()

        while True:
            if frame == 2:
                vis.add_geometry(point_list)

            vis.update_geometry(point_list)
            vis.poll_events()
            vis.update_renderer()
            time.sleep(0.005)

            world.tick()

            # We here add a spectator to watch how our ego vehicle will move
            spectator = world.get_spectator()
            transform = vehicle.get_transform()  # we get the transform of vehicle
            spectator.set_transform(carla.Transform(transform.location + carla.Location(z=50),
                                                    carla.Rotation(pitch=-90)))

            process_time = datetime.now() - dt0
            sys.stdout.write("\r" + "FPS: " + str(1.0 / process_time.total_seconds()) + "Current Frame: " + str(frame))
            sys.stdout.flush()
            dt0 = datetime.now()

            frame += 1

    finally:
        world.apply_settings(original_settings)
        traffic_manager.set_synchronous_mode(False)
        vehicle.destroy()
        lidar.destroy()
        vis.destroy_window()

In [15]:
np.concatenate([np.array([[], [], []]), np.array([[1, 2], [2, 3], [3, 4]])], axis=1).shape

(3, 2)

In [9]:
import numpy as np
np.empty((3, 0), dtype=np.float64)

array([], shape=(3, 0), dtype=float64)

In [12]:
spawn_points[0]

In [ ]:
from carla_patch import CarlaPatch
import torch
import numpy as np
patch = torch.load('C:/Users/Kai/Desktop/patch.pt', map_location='cpu')
carla_patch = CarlaPatch(patch, patch_location=(0, -0, 0), patch_rotation=(70, 60, 40), pixel_length=0.5)
patch_data, pos_3d = carla_patch()
# pos_3d = pos_3d[:3].transpose(0, 2).reshape(-1, 3).numpy()
# patch_data = patch_data.transpose(0, 2).reshape(-1, 3).detach_().numpy()
patch_data = patch_data.permute((1, 2, 0)).detach().numpy()
# p3d, color = prj_patch2point_cloud(prj_config[0], patch_data, prj_config[1], prj_config[2])

In [4]:
import numpy as np
import open3d as o3d
# from pyntcloud import PyntCloud

# points = pos_3d[:3].numpy().reshape((-1, 3), order='F')
points = np.concatenate(point_clouds[0: 2], axis=1)[:3].transpose()
points_color = np.concatenate([colors[0], np.zeros_like(colors[1])], axis=0)
# points = point_clouds[81][:3].transpose()
point_cloud = o3d.geometry.PointCloud()
point_cloud.points = o3d.utility.Vector3dVector(np.array(points))
point_cloud.colors = o3d.utility.Vector3dVector(points_color / 255.0)
# point_cloud.farthest_point_down_sample(num_samples=1000)
o3d.visualization.draw_geometries([point_cloud])

In [ ]:
p3d[2] *= -1

In [5]:
# points = pos_3d[:3].numpy().reshape((-1, 3), order='F')
points = np.concatenate(point_clouds[0: 5], axis=1)[:3].transpose()
points_color = np.concatenate(colors[0: 5], axis=0)
# points = point_clouds[81][:3].transpose()
point_cloud = o3d.geometry.PointCloud()
point_cloud.points = o3d.utility.Vector3dVector(np.array(points))
point_cloud.colors = o3d.utility.Vector3dVector(points_color / 255.0)
# point_cloud.farthest_point_down_sample(num_samples=1000)
o3d.visualization.draw_geometries([point_cloud])

In [ ]:
color.shape

In [ ]:
# points = pos_3d[:3].numpy().reshape((-1, 3), order='F')
# points = point_clouds[81][:3].transpose()
point_cloud = o3d.geometry.PointCloud()
point_cloud.points = o3d.utility.Vector3dVector(np.array(p3d).transpose())
point_cloud.colors = o3d.utility.Vector3dVector(color)
# point_cloud.farthest_point_down_sample(num_samples=1000)
o3d.visualization.draw_geometries([point_cloud])

In [8]:
points = np.concatenate(point_clouds[:], axis=1)[:3].transpose()
points_color = np.concatenate(colors[:], axis=0)
pcd = o3d.geometry.PointCloud()#传入3d点云
pcd.points = o3d.utility.Vector3dVector(np.array(points))	#point3D二维numpy矩阵,将其转换为open3d点云格式
pcd.colors = o3d.utility.Vector3dVector(points_color / 255.0)

vis = o3d.visualization.Visualizer()
vis.create_window()	#创建窗口
render_option: o3d.visualization.RenderOption = vis.get_render_option()	#设置点云渲染参数
render_option.background_color = np.array([0, 0, 0])	#设置背景色（这里为黑色）
render_option.point_size = 0.5	#设置渲染点的大小

vis.add_geometry(pcd)	#添加点云
vis.run()
vis.destroy_window()

In [ ]:
vis.destroy_window()